# Packages

In [20]:
import pyspark
from pyspark.sql import SparkSession, functions as F

In [4]:
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('SQL') \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 09:33:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load files

In [5]:
green_df = spark.read.parquet('/Users/gonzo/Desktop/de_bootcamp/week_5/data/green/processed/*/*/*')

In [6]:
yellow_df = spark.read.parquet('/Users/gonzo/Desktop/de_bootcamp/week_5/data/yellow/processed/*/*/*')

# Change column names

In [11]:
green_df = green_df.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                   .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [13]:
yellow_df = yellow_df.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                   .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [12]:
green_df.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [14]:
yellow_df.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

# Common columns

In [17]:
yellow_col = set(yellow_df.columns)
common_col = [col for col in green_df.columns if col in yellow_col]

In [18]:
common_col

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

# Combine the two Spark Dataframes

In [21]:
green_df_type = green_df.select(common_col) \
                        .withColumn('service_type', F.lit('green'))

In [25]:
yellow_df_type = yellow_df.select(common_col) \
                        .withColumn('service_type', F.lit('yellow'))

In [26]:
df_trips_data = green_df_type.unionAll(yellow_df_type)

In [27]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



# Spark SQL

In [30]:
df_trips_data.createOrReplaceTempView('trips_data')

In [36]:
df_result = spark.sql \
("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [38]:
df_result.coalesce(1).write.parquet('/Users/gonzo/Desktop/de_bootcamp/week_5/reports/revenue')